In [167]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# ___library_import_statements___
import pandas as pd
# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
#import datetime
import numpy as np


In [168]:
#newest yahoo API 
!pip install yfinance
import yfinance as yf
%matplotlib inline

In [ ]:
#start_time = datetime.datetime(2017, 10, 1)
#end_time = datetime.datetime(2019, 1, 20)
#end_time = datetime.datetime.now().date().isoformat()         # today

In [169]:
# yahoo gives only daily historical data
def get_data(ticker):
    connected = False
    while not connected:
        try:
            df = yf.download(ticker, period="5y", interval="1d", group_by = 'ticker', auto_adjust = False)
            connected = True
            print('connected to yahoo')
        except Exception as e:
            print("type error: " + str(e))
            time.sleep( 5 )
            pass   
    return df    

In [170]:
def computeRSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [171]:
result = pd.DataFrame(index=np.arange(0, 50), columns=['Date', 'Ticker', 'Close', 'RSI'])

# ___variables___
ticker = ['AMZN',
          'NVDA',
          'BABA',
          '9988.HK',                                               
          '0700.HK',
          '2382.HK',
          '3690.HK',
          'MELI',
          'OKTA',
          'SPOT',
          'TWLO',
          'DDOG',
          'MDB',
          'TXG',
          'CRWD',
          'SE',
          'BZUN',
          '9991.HK',
          '2158.HK',
          '0909.HK',
          '9698.HK',
          '600276.SS',
          '83188.HK',
          'UL',
          '0066.HK',
          '0002.HK',
          '0003.HK',
          '0270.HK',
          '0823.HK',
          '1044.HK',
          '0941.HK',
          '0001.HK',
          '2318.HK',
          '0016.HK',
          '3141.HK',
          '83199.HK',
          '9187.HK'
         ]

In [178]:
for i in range(len(ticker)):

    # run the function
    df = get_data(ticker[i])   

    # resample with dictionary
    # each column has to be handled differently
    agg_dict = {'Open': 'first',
              'High': 'max',
              'Low': 'min',
              'Close': 'last',
              'Adj Close': 'last',
              'Volume': 'mean'}

    r_df = df.resample('W').agg(agg_dict)

    r_df['RSI'] = computeRSI(r_df['Close'], 14)

    result.loc[i] = [r_df.index[-1].date(), 
                     ticker[i], 
                     "%.2f" % r_df['Close'][-1], 
                     round(r_df['RSI'][-1], 0)]

In [179]:
result